In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
%matplotlib inline

In [25]:
%run ImageDataLoader.ipynb
%run ImageProcessor.ipynb
%run DataExplorer.ipynb
%run DatasetStatistics.ipynb
%run DuplicateDetector.ipynb
%run OversampledDataset.ipynb
%run BatchVisualizer.ipynb
%run VGG16Model.ipynb
%run ResNet34Model.ipynb
%run EfficientNetModel.ipynb
%run Trainer.ipynb

In [3]:
DATA_ROOT = "./data"
TRAIN_DIR = "./data/Training"
TEST_DIR = "./data/Testing"
BATCH_SIZE = 64
IMAGE_SIZE = (224, 224)
VAL_SPLIT = 0.15

In [ ]:
device = (
    torch.accelerator.current_accelerator().type
    if torch.accelerator.is_available()
    else "cpu"
)
print(f"Using {device} device")
print(f"Accelerator name: {torch.cuda.get_device_name(device)}")

In [ ]:
loader = ImageDataLoader(TRAIN_DIR, TEST_DIR)
all_files = loader.load_all_images()

print(f"Successfully loaded {len(all_files)} images")

In [ ]:
duplicate_detector = DuplicateDetector(all_files)
duplicate_detector.detect_duplicates()

if len(duplicate_detector.duplicates) > 0:  # If duplicate files are present
    duplicate_detector.remove_duplicates_from_disk()  # Removing duplicates entirely from disk
    all_files = duplicate_detector.get_unique_files()  # Cleaning list with file paths

In [ ]:
loader.print_dataset_class_count()

In [ ]:
processor = ImageProcessor(all_files)

processor.load_grayscale_images()
processor.display_image_grid(batch_size=32, figsize=(18, 9), images_per_row=8)

In [ ]:
explorator = DataExplorer(all_files)
explorator.retrieve_sample_of_images(
    [0, len(all_files) // 2, -1]
)  # First, middle and last image

explorator.plot_histogram()

In [ ]:
stats = DatasetStatistics(processor.gray_images)

stats.compute_stats()
stats.print_stats()

MEAN, STD = stats.get_normalized_values()

In [11]:
train_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(12),
        transforms.ColorJitter(brightness=0.15, contrast=0.15),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [12]:
test_transform = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=[MEAN] * 3, std=[STD] * 3),
    ]
)

In [13]:
base_dataset = datasets.ImageFolder(TRAIN_DIR)

ind = list(range(len(base_dataset)))
targets = [base_dataset.targets[i] for i in ind]

train_idx, val_idx = train_test_split(
    ind, test_size=VAL_SPLIT, random_state=42, stratify=targets
)

In [14]:
val_base = datasets.ImageFolder(TRAIN_DIR, transform=test_transform)
valset = Subset(val_base, val_idx)

In [ ]:
trainset = OversampledDataset(TRAIN_DIR, transform=train_transform, indices=train_idx)
testset = datasets.ImageFolder(TEST_DIR, transform=test_transform)

print()
trainset.print_class_distribution()

print()
print(f"Training samples: {len(trainset)}")
print(f"Validation samples: {len(valset)}")
print(f"Test samples: {len(testset)}")
print(f"Classes: {trainset.classes}")

In [27]:
train_dl = DataLoader(trainset, BATCH_SIZE, shuffle=True, num_workers=0)
val_dl = DataLoader(valset, BATCH_SIZE, shuffle=False, num_workers=0)
test_dl = DataLoader(testset, BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
visualizer = BatchVisualizer(trainset.classes, mean=[MEAN] * 3, std=[STD] * 3)

visualizer.visualize_batch(train_dl)
visualizer.visualize_classes(val_dl, 6)

In [29]:
EPOCHS = 5
LEARNING_RATES = [0.01, 0.001, 0.0001, 0.00001]
NUMBER_OF_CLASSES = 4

In [30]:
models_config = {
    "VGG-16": VGG16Model,
    "ResNet34": ResNet34Model,
    "EfficientNet_B0": EfficientNetModel,
}

In [31]:
optimizers_config = {
    "sgd": lambda params, lr: optim.SGD(params, lr=lr),
    "sgd_momentum": lambda params, lr: optim.SGD(params, lr=lr, momentum=0.9),
    "adam": lambda params, lr: optim.Adam(params, lr=lr),
}

In [32]:
def set_seed(seed=42):
    random.seed(seed)  # Python random
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (used)
    torch.cuda.manual_seed_all(seed)  # PyTorch all GPUs
    torch.backends.cudnn.deterministic = True  # CUDA deterministic operations
    torch.backends.cudnn.benchmark = False  # Turn off autotuning for reproducity

In [33]:
def run_experiment(
    model_class, optimizer_fn, lr, train_dl, val_dl, device, epochs=EPOCHS
):
    set_seed(42)

    model = model_class(NUMBER_OF_CLASSES)
    trainer = Trainer(model, device)
    optimizer = optimizer_fn(model.parameters(), lr)

    history = trainer.fit(train_dl, val_dl, optimizer, epochs)

    return {
        "last_epoch_val_acc": history["val_acc"][-1],
        "best_val_acc": max(history["val_acc"]),
        "last_epoch_val_loss": history["val_loss"][-1],
        "history": history,
    }

In [34]:
def run_all_experiments(train_dl, val_dl, device):
    res = []

    total = len(models_config) * len(optimizers_config) * len(LEARNING_RATES)
    current = 0

    for model_name, model_class in models_config.items():
        for optim_name, optim_fn in optimizers_config.items():
            for lr in LEARNING_RATES:
                current += 1
                print(f"\n[{current}/{total}] {model_name} + {optim_name} + lr={lr}")

                result = run_experiment(
                    model_class, optim_fn, lr, train_dl, val_dl, device
                )

                res.append(
                    {
                        "model": model_name,
                        "optimizer": optim_name,
                        "learning rate": lr,
                        **result,
                    }
                )
                print(
                    f"Last epoch's validation accuracy: {result['last_epoch_val_acc']:.4f}"
                )

    return res

In [35]:
results = run_all_experiments(train_dl, val_dl, device)


[1/36] VGG-16 + sgd + lr=0.01
Epoch 1 out of 5 epochs.                   	Train loss: 0.9347, train accuracy: 0.6246                   	Validation loss: 0.6848, validation accuracy: 0.7266
Epoch 2 out of 5 epochs.                   	Train loss: 0.4978, train accuracy: 0.8189                   	Validation loss: 0.5281, validation accuracy: 0.7866
Epoch 3 out of 5 epochs.                   	Train loss: 0.3932, train accuracy: 0.8567                   	Validation loss: 0.4755, validation accuracy: 0.8225
Epoch 4 out of 5 epochs.                   	Train loss: 0.2883, train accuracy: 0.8968                   	Validation loss: 0.3225, validation accuracy: 0.8897
Epoch 5 out of 5 epochs.                   	Train loss: 0.2596, train accuracy: 0.9096                   	Validation loss: 0.3159, validation accuracy: 0.8729
Training completed in 1.82 minute(s)
Last epoch's validation accuracy: 0.8729

[2/36] VGG-16 + sgd + lr=0.001
Epoch 1 out of 5 epochs.                   	Train loss: 1.2106, 

In [36]:
FINAL_EPOCHS = 50
PATIENCE = 10

In [38]:
set_seed(42)

vgg16_model = VGG16Model()
optimizer_vgg16 = optim.Adam(vgg16_model.parameters(), lr=0.0001)

trainer_vgg16 = Trainer(vgg16_model, device)
vgg16_history = trainer_vgg16.fit(
    train_dl, val_dl, optimizer_vgg16, FINAL_EPOCHS, PATIENCE
)
vgg16_acc, vgg16_preds, vgg16_labels = trainer_vgg16.test(test_dl)
print(f"Test accuracy: {vgg16_acc}")

Epoch 1 out of 50 epochs.                   	Train loss: 0.6579, train accuracy: 0.7497                   	Validation loss: 0.3552, validation accuracy: 0.8633
Epoch 2 out of 50 epochs.                   	Train loss: 0.2915, train accuracy: 0.8900                   	Validation loss: 0.2900, validation accuracy: 0.8921
Epoch 3 out of 50 epochs.                   	Train loss: 0.1576, train accuracy: 0.9444                   	Validation loss: 0.2176, validation accuracy: 0.9233
Epoch 4 out of 50 epochs.                   	Train loss: 0.1062, train accuracy: 0.9629                   	Validation loss: 0.3804, validation accuracy: 0.9017
Epoch 5 out of 50 epochs.                   	Train loss: 0.0738, train accuracy: 0.9720                   	Validation loss: 0.1343, validation accuracy: 0.9568
Epoch 6 out of 50 epochs.                   	Train loss: 0.0777, train accuracy: 0.9766                   	Validation loss: 0.1921, validation accuracy: 0.9305
Epoch 7 out of 50 epochs.               

In [50]:
set_seed(42)

vgg16_model = VGG16Model(freeze_features=True)
optimizer_vgg16 = optim.Adam(vgg16_model.parameters(), lr=0.0001)

trainer_vgg16 = Trainer(vgg16_model, device)
vgg16_history = trainer_vgg16.fit(
    train_dl, val_dl, optimizer_vgg16, FINAL_EPOCHS, PATIENCE
)
vgg16_acc, vgg16_preds, vgg16_labels = trainer_vgg16.test(test_dl)
print(f"Test accuracy: {vgg16_acc}")

Epoch 1 out of 50 epochs.                   	Train loss: 0.6458, train accuracy: 0.7414                   	Validation loss: 0.5085, validation accuracy: 0.7890
Epoch 2 out of 50 epochs.                   	Train loss: 0.3483, train accuracy: 0.8684                   	Validation loss: 0.4718, validation accuracy: 0.8129
Epoch 3 out of 50 epochs.                   	Train loss: 0.2604, train accuracy: 0.9025                   	Validation loss: 0.4206, validation accuracy: 0.8225
Epoch 4 out of 50 epochs.                   	Train loss: 0.2319, train accuracy: 0.9104                   	Validation loss: 0.2920, validation accuracy: 0.8873
Epoch 5 out of 50 epochs.                   	Train loss: 0.1866, train accuracy: 0.9316                   	Validation loss: 0.3131, validation accuracy: 0.8873
Epoch 6 out of 50 epochs.                   	Train loss: 0.1799, train accuracy: 0.9323                   	Validation loss: 0.4773, validation accuracy: 0.8297
Epoch 7 out of 50 epochs.               

In [40]:
set_seed(42)

resnet34_model = ResNet34Model()
optimizer_resnet = optim.SGD(resnet34_model.parameters(), lr=0.001, momentum=0.9)

trainer_resnet34 = Trainer(resnet34_model, device)
resnet34_history = trainer_resnet34.fit(
    train_dl, val_dl, optimizer_resnet, FINAL_EPOCHS, PATIENCE
)
resnet34_acc, resnet34_preds, resnet34_labels = trainer_resnet34.test(test_dl)
print(f"Test accuracy: {resnet34_acc}")

Epoch 1 out of 50 epochs.                   	Train loss: 0.8898, train accuracy: 0.6612                   	Validation loss: 0.5351, validation accuracy: 0.8129
Epoch 2 out of 50 epochs.                   	Train loss: 0.3674, train accuracy: 0.8764                   	Validation loss: 0.4227, validation accuracy: 0.8393
Epoch 3 out of 50 epochs.                   	Train loss: 0.2396, train accuracy: 0.9157                   	Validation loss: 0.2910, validation accuracy: 0.8825
Epoch 4 out of 50 epochs.                   	Train loss: 0.1925, train accuracy: 0.9323                   	Validation loss: 0.2125, validation accuracy: 0.9281
Epoch 5 out of 50 epochs.                   	Train loss: 0.1209, train accuracy: 0.9641                   	Validation loss: 0.2051, validation accuracy: 0.9281
Epoch 6 out of 50 epochs.                   	Train loss: 0.0935, train accuracy: 0.9713                   	Validation loss: 0.1617, validation accuracy: 0.9448
Epoch 7 out of 50 epochs.               

In [41]:
set_seed(42)

efficientnet_model = EfficientNetModel()
optimizer_efficientnet = optim.Adam(efficientnet_model.parameters(), lr=0.001)

trainer_efficientnet = Trainer(efficientnet_model, device)
efficientnet_history = trainer_efficientnet.fit(
    train_dl, val_dl, optimizer_efficientnet, FINAL_EPOCHS, PATIENCE
)
efficientnet_acc, efficientnet_preds, efficientnet_labels = trainer_efficientnet.test(
    test_dl
)
print(f"Test accuracy: {efficientnet_acc}")

Epoch 1 out of 50 epochs.                   	Train loss: 0.4560, train accuracy: 0.8318                   	Validation loss: 0.2140, validation accuracy: 0.9281
Epoch 2 out of 50 epochs.                   	Train loss: 0.1863, train accuracy: 0.9335                   	Validation loss: 0.1834, validation accuracy: 0.9376
Epoch 3 out of 50 epochs.                   	Train loss: 0.0990, train accuracy: 0.9648                   	Validation loss: 0.3706, validation accuracy: 0.8897
Epoch 4 out of 50 epochs.                   	Train loss: 0.0925, train accuracy: 0.9671                   	Validation loss: 0.2426, validation accuracy: 0.9233
Epoch 5 out of 50 epochs.                   	Train loss: 0.0604, train accuracy: 0.9803                   	Validation loss: 0.0864, validation accuracy: 0.9736
Epoch 6 out of 50 epochs.                   	Train loss: 0.0480, train accuracy: 0.9860                   	Validation loss: 0.1170, validation accuracy: 0.9544
Epoch 7 out of 50 epochs.               

In [42]:
def plot_train_val_accuracy(train_acc, val_acc, title=None, ax=None):
    if ax is None:
        _, ax = plt.subplots()

    ax.plot(train_acc, "r--", label="Training accuracy")
    ax.plot(val_acc, "b-", label="Validation accuracy")

    ax.set_xlabel("Epoch")
    ax.set_ylabel("Accuracy")
    if title is not None:
        ax.set_title(title, fontsize=10)

    ax.grid()
    ax.legend()

In [ ]:
_, axs = plt.subplots(ncols=3, figsize=(15, 5))
plot_train_val_accuracy(
    vgg16_history["train_acc"], vgg16_history["val_acc"], "VGG-16 model", axs[0]
)

plot_train_val_accuracy(
    resnet34_history["train_acc"], resnet34_history["val_acc"], "ResNet34 model", axs[1]
)

plot_train_val_accuracy(
    efficientnet_history["train_acc"],
    efficientnet_history["val_acc"],
    "EfficientNet model",
    axs[2],
)

plt.suptitle("Models training accuracy vs validation accuracy", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()